## Handling of results from a single run of the simulation

In [94]:
# Import libraries
import pandas as pd
import json
import os 
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [96]:
import shlex

# Settings
# model = "gp"
model = "anthropic/claude-sonnet-4"
temperature = 1
samples_generated = 10
feedback_iterations = 10
one_shot = True
spec_type = "both"  # ["natural", "formal", "both"]

# Inputs
formal_specification_name = "formal_specification.h"
natural_language_specification_name = "natural_language_specification.h"
function_signature_name = "function_signature.c"
extra_code_name = "extras.c"
test_cases_name = "solution_test.c"

prompt_type = "one-shot" if one_shot else "zero-shot"
output_folder_name = f"{model}-{samples_generated}-{feedback_iterations}-{temperature}-{prompt_type}-{spec_type}"

problems_directory = "../journal_problems/"
wp_model = "real"
cmd = [
    "python3", "main.py", "generate_code_folder",
    "-d", "../journal_problems/",
    "-samples", str(samples_generated),
    "-iter", str(feedback_iterations),
    "-temp", str(temperature),
    "-wpt", "10",
    "-o", f"../output/{output_folder_name}",
    "-of", "generated_code.c",
    "-fsf", formal_specification_name,          
    "-nl", natural_language_specification_name, 
    "-sig", function_signature_name,            
    "-pt", prompt_type,                         
    "-spectype", spec_type,                     
    "-model", model,                            
    "-tc", test_cases_name,                     
]

if extra_code_name and extra_code_name.strip():
    cmd += ["-extra", extra_code_name]

if wp_model:
    cmd += ["-wpm", wp_model]

command = " ".join(shlex.quote(x) for x in cmd)

print(f"output_folder: {output_folder_name}")
print(command)


output_folder: anthropic/claude-sonnet-4-10-10-1-one-shot-both
python3 main.py generate_code_folder -d ../journal_problems/ -samples 10 -iter 10 -temp 1 -wpt 10 -o ../output/anthropic/claude-sonnet-4-10-10-1-one-shot-both -of generated_code.c -fsf formal_specification.h -nl natural_language_specification.h -sig function_signature.c -pt one-shot -spectype both -model anthropic/claude-sonnet-4 -tc solution_test.c -extra extras.c -wpm real


In [97]:
# print all folders in the directory sorted by name
folders = os.listdir(f'../output/{output_folder_name}')

# Sort on number
folders.sort(key=lambda x: int(x.split('-')[0]))

# Create an empty dataframe that will be filled with info of the iterations
iteration_array = []

# Filter the folders if need be, only take ones higher than 300
# folders = [folder for folder in folders if int(folder.split('-')[0]) >= 427]

total_completions_used = []
total_tokens_used = []
total_completions = []
verification_time = []

# For each folder read the output and add it to the dataframe
for folder_name in folders:
    # Read the output given in the file 
    with open(f"../output/{output_folder_name}/{folder_name}/results.txt", 'r') as file:
        # Read the file which contains an array
        data = json.load(file)

    verified_goals = []
    passed_tests = []
    verified_goals_percentage = []
    passed_tests_percentage = []
    generated_code_length = []
    
    total_completions_used.append(data['total_completions_used'])
    total_tokens_used.append(data['total_tokens_used'])
    total_completions.append(data['total_completions_requested'])
    verification_time.append(data['total_time_taken_verification'])
    
    # Get all completions in the initial code generation and in each code improvement iteration
    completions = []
    for i in data["initial_code_generation_information"][0]['completions']:
        completions.append(i)
    
    for i in data["code_improvement_information"]:
        for j in i['completions']:
            completions.append(j)

    # for each iteration in the array add it to the dataframe
    for i in completions:
        verified_goals.append(i['verified_goals_count'])

        # If the goals is 0 / 0 the percentage is 0
        if (i['verified_goals_count'] == "0 / 0"):
            verified_goals_percentage.append(0)
        else:
            verified_goals_percentage.append(eval(i['verified_goals_count']))
        # If there is a compilation error the length is only 1
        if (len(i["test_information"]) == 1):
            passed_tests.append(f"{i['test_information']['summary']['passed']} / {i['test_information']['summary']['total']}")
            passed_tests_percentage.append(0)
        else:
            passed_tests.append(f"{i['test_information'][-1]['summary']['passed']} / {i['test_information'][-1]['summary']['total']}")
            passed_tests_percentage.append(i['test_information'][-1]['summary']['passed'] / i['test_information'][-1]['summary']['total'])

        # Get the length of the generated code at this iteration by counting the newlines
        generated_code_length.append(i['gpt_output'].count("\n"))

    # Add it as a column to the dataframe
    iteration_array.append(verified_goals)
    iteration_array.append(passed_tests)
    iteration_array.append(verified_goals_percentage)
    iteration_array.append(passed_tests_percentage)
    iteration_array.append(generated_code_length)

# Create a dataframe with the information of the iterations
df = pd.DataFrame(iteration_array)


## Work together with google sheets and google drive to store the information and results


In [98]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ---- auth ----
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name(
    "../tmp/vecogen-3f08ff3c7219.json", SCOPES
)
client = gspread.authorize(creds)

# ---- spreadsheet + template ----
SPREADSHEET_ID = "1fTotOpVTfxCr84Wg2inBKAn-ELfgLQMkM8qnmRT_UQE"
sheet = client.open_by_key(SPREADSHEET_ID)

template_sheet_descriptive_name = "result_template"
template_ws = sheet.worksheet(template_sheet_descriptive_name)

# ---- duplicate template as last sheet with new name ----
new_sheet_name = output_folder_name  # assume defined
sheet.duplicate_sheet(
    source_sheet_id=template_ws.id,
    insert_sheet_index=len(sheet.worksheets()),  # last position
    new_sheet_name=new_sheet_name,
)

worksheet = sheet.worksheet(new_sheet_name)

# ---- write the main table starting at R50 (auto-sizes to df) ----
start_cell = "R50"
data_to_update = df.values.tolist()
worksheet.update(start_cell, data_to_update)

# ---- write single cells ----
worksheet.update_acell("C3",  model)
worksheet.update_acell("C4",  temperature)
worksheet.update_acell("C5",  samples_generated)
worksheet.update_acell("C6",  feedback_iterations)
worksheet.update_acell("C7",  spec_type)
worksheet.update_acell("C8",  command)
worksheet.update_acell("C9",  sum(verification_time))
worksheet.update_acell("C11", prompt_type)

# ---- write the extra block ----
total_extra_data = []
for i in range(len(total_tokens_used)):
    total_extra_data.append([total_tokens_used[i], total_completions_used[i], total_completions[i], verification_time[i]])
    total_extra_data += [["", "", "", ""]] * 4

worksheet.update("L50:O424", total_extra_data)


C:\Users\MSEGEZ\AppData\Local\Temp\ipykernel_39212\1976603367.py:34: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(start_cell, data_to_update)
C:\Users\MSEGEZ\AppData\Local\Temp\ipykernel_39212\1976603367.py:52: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("L50:O424", total_extra_data)


{'spreadsheetId': '1fTotOpVTfxCr84Wg2inBKAn-ELfgLQMkM8qnmRT_UQE',
 'updatedRange': "'anthropic/claude-sonnet-4-10-10-1-one-shot-both'!L50:O374",
 'updatedRows': 325,
 'updatedColumns': 4,
 'updatedCells': 1300}

## Store the data in the google drive folder    

In [ ]:
len(total_tokens_used)

65

In [ ]:
# Define the scope and authenticate using service account credentials
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file('../tmp/vecogen-3f08ff3c7219.json', scopes=SCOPES)

# Authenticate Google Drive API client
drive_service = build('drive', 'v3', credentials=creds)

def create_folder(name, parent_id=None):
    """Create a folder in Google Drive."""
    folder_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    if parent_id:
        folder_metadata['parents'] = [parent_id]
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    return folder.get('id')

def upload_file(file_path, parent_id):
    """Upload a file to Google Drive."""
    file_name = os.path.basename(file_path)
    media = MediaFileUpload(file_path, resumable=True)
    file_metadata = {
        'name': file_name,
        'parents': [parent_id]
    }
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file.get('id')

def upload_folder(local_folder_path, parent_id=None):
    """Upload a folder to Google Drive, recursively including all files and subfolders."""
    folder_name = os.path.basename(local_folder_path)
    folder_id = create_folder(folder_name, parent_id)
    for item in os.listdir(local_folder_path):
        item_path = os.path.join(local_folder_path, item)
        if os.path.isdir(item_path):
            upload_folder(item_path, folder_id)
        else:
            upload_file(item_path, folder_id)

def list_folders():
    """List all folders in Google Drive."""
    query = "mimeType='application/vnd.google-apps.folder' and trashed=false"
    results = drive_service.files().list(q=query, spaces='drive', fields='nextPageToken, files(id, name)').execute()
    folders = results.get('files', [])

    if not folders:
        print('No folders found.')
    else:
        print('Folders:')
        for folder in folders:
            print(f"Name: {folder['name']}, ID: {folder['id']}")

# Define the local folder you want to upload and the Google Drive parent folder ID
local_folder_path = f'../output/{output_folder_name}'
drive_parent_folder_id = '1ZvsBlLV94f1TTG-2aIfeX_v86I4bOxq8'

# Start the folder upload process
upload_folder(local_folder_path, drive_parent_folder_id)

print(f'Folder {local_folder_path} uploaded to Google Drive folder with ID {drive_parent_folder_id}')

# Put the link in C10
worksheet.update_acell('C10', f"https://drive.google.com/drive/folders/{drive_parent_folder_id}")


Folder ../output/gpt-5-10-10-1-one-shot-both uploaded to Google Drive folder with ID 1ZvsBlLV94f1TTG-2aIfeX_v86I4bOxq8


{'spreadsheetId': '1fTotOpVTfxCr84Wg2inBKAn-ELfgLQMkM8qnmRT_UQE',
 'updatedRange': "'gpt-4o-1-10-0-one-shot-both'!C10",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}